# Making altair plots to save to Jekyll

## Using altair

### Pre-plot setup

In [17]:
import pandas as pd
import altair as alt

In [18]:
url = "https://github.com/UIUC-iSchool-DataViz/is445_data/raw/main/licenses_fall2022.csv"
df = pd.read_csv(url)

In [19]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

## Plot 1 - static

In [20]:
license_counts = (
    df['License Type']
    .value_counts()
    .reset_index(name='COUNT')
    .rename(columns={'index': 'License Type'})
    .head(10)
)

plot1 = alt.Chart(license_counts).mark_bar().encode(
    x=alt.X('COUNT:Q', title='Number of Licenses'),
    y=alt.Y('License Type:N', sort='-x', title='License Type'),
    color=alt.Color('License Type:N', legend=None)
).properties(
    width=600,
    height=400,
    title='Top 10 Most Common Professional License Types'
)

plot1

alt.Chart(...)

In [22]:
myJekyllDir = 'C:/Users/1234/alisharawat56.github.io/assets/json/'

In [23]:
plot1.save(myJekyllDir + 'hw_static.json')

## Plot 2 - interactive

In [30]:
df_clean = df[['License Type', 'Zip']].dropna()
df_clean = df_clean[df_clean['Zip'] != 0]
df_clean['Zip'] = df_clean['Zip'].astype(str)

# Get top 40 Zip Codes by total count
top_zips = (
    df_clean.groupby('Zip').size()
    .sort_values(ascending=False)
    .head(40)
    .index
)

df_top = df_clean[df_clean['Zip'].isin(top_zips)]

# Aggregate counts
license_by_zip = df_top.groupby(['License Type', 'Zip']).size().reset_index(name='COUNT')

# Brush selection
brush = alt.selection_interval(encodings=['x', 'y'])

# Heatmap
heatmap = alt.Chart(license_by_zip).mark_rect().encode(
    x=alt.X('Zip:N', sort='-y', title='Zip Code'),
    y=alt.Y('License Type:N', sort='-x', title='License Type'),
    color=alt.Color('COUNT:Q', title='Number of Licenses', scale=alt.Scale(scheme='blues')),
    tooltip=['License Type', 'Zip', 'COUNT']
).add_params(
    brush
).properties(
    width=375,
    height=450,
    title='License Type × Zip Code (Top 40 Zips) – Drag to Select'
)

# Linked bar chart - simpler, sorted, one color
bars = alt.Chart(license_by_zip).mark_bar(color='#4C72B0').encode(
    y=alt.Y('Zip:N', sort='-x', title='Zip Code'),
    x=alt.X('COUNT:Q', title='Licenses in Selected Area'),
    tooltip=['Zip', 'COUNT']
).transform_filter(
    brush
).properties(
    width=230,
    height=450,
    title='Licenses for Selected Area'
)

plot2 = heatmap | bars
plot2

alt.HConcatChart(...)

In [24]:
plot2.save(myJekyllDir + 'hw_interactive.json')